In [560]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity,pairwise_kernels
import math
import dask.dataframe as dd
from scipy import spatial

In [561]:
df = pd.read_csv('./Data/VideoStarted.csv')

In [562]:
df = df.iloc[:10000]

# Data Manipulation

In [563]:
#Combining the Date, Minute_Of_Day, Second to get a timestamp
def make_timestamp(row):
    date_as_string = str(row['Date'])
    hour = str(row['Minute_Of_Day'] // (60))
    minute = str(row['Minute_Of_Day'] % 60)
    second = str(row['Second'])
    
    if len(hour) == 1:
        hour = '0' + hour
        
    if len(minute) == 1:
        minute = '0' + minute
    
    if len(second) == 1:
        second = '0' + second
    
    date_string = date_as_string[:4] + '-' + date_as_string[4:6] + '-' + date_as_string[6:] + ' ' + hour + ':' + minute + ':' + second
    datetime_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')
    return datetime_object

In [564]:
df['TimeStamp'] = df.apply(make_timestamp, axis=1)

In [565]:
#Changing timestamp to the epoch value 
def change_timestamp_to_epoch(row):
    pattern = '%Y-%m-%d %H:%M:%S'
    timestamp = str(row['TimeStamp'])
    epoch = int(time.mktime(time.strptime(timestamp, pattern)))
    return epoch

In [566]:
df['epoch'] = df.apply(change_timestamp_to_epoch,axis=1)

In [567]:
#Taking the epoch time of the 50th percentile as the start date and the next two days as the end date 
#for prediction
epoch_start_date = df['epoch'].quantile(0.5)
epoch_end_date = epoch_start_date+48*60*60*1000

In [568]:
#Scaling the epoch column
scaler = MinMaxScaler()
df['epoch'] = scaler.fit_transform(df[['epoch']])
df['epoch'].describe()

C:\Users\rusha\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


count    10000.000000
mean         0.552504
std          0.275514
min          0.000000
25%          0.344181
50%          0.518322
75%          0.834544
max          1.000000
Name: epoch, dtype: float64

In [569]:
#Scaling the start_date, end_date
epoch_start_date = pd.DataFrame(index=[0],data={'start_date': epoch_start_date})
epoch_end_date = pd.DataFrame(index=[0],data={'end_date': epoch_end_date})
epoch_start_date = scaler.transform(epoch_start_date)
epoch_end_date = scaler.transform(epoch_end_date)
epoch_start_date,epoch_end_date

(array([[0.51832232]]), array([[1141.86574637]]))

In [570]:
#Finding the number of unique users
len(df['UserId'].unique())

2430

In [571]:
#Finding the number of unique videos
len(df['VideoId'].unique())

691

# User-Id vs Video-Id Matrix

In [572]:
#Finding the different videos seen by a user
x =  df[:100].groupby(['UserId', 'VideoId'])
x = x.first()
x

Date  \
UserId                           VideoId                                      
0280dfdd112732a3ac12b12dc770b7af 36a27b379622f342ec87f9aafadb8f94  20170106   
04c1e14fd54a06c0d4fe98cf0cb8b04c 806660cb47633263a24bbc53238a9a53  20170106   
057212548aff678c01fe2a55224cc2cc f3ecc88c6e53a990a75bd7e9b7ddb06e  20170106   
079ec44549c738db2d55ff44b5b139f3 66ea27601d482d1857f6977450b2dc1c  20170106   
087c741c167a352d07822fcab58046e5 2e47c28c5a2f4ac65315d0229447fdf8  20170106   
0d164b3342b7c442b8a5cfc8a608bee1 afb85ac1bc0bb04bd7227ff5bf925bad  20170106   
                                 d0c64e942ff427750850e49894ec48fe  20170106   
0d2717ba754737c78430788ed77a1db7 675548e120ccb01387cc3ca9fed870a9  20170106   
13d9f55e297d7d9c219fbe9085e1006c 8b6076b28243980a22f496bb127f3aec  20170106   
16455c25d7a8af4084973ec08d320cd5 99e672aa585e8af17d01da8b839b6d18  20170106   
19859f9b1e56c687b010391bc6670a76 57622239c0d0c605eb50e6923ebabb8a  20170106   
1f9be9226d4393fff4b8d6b94e54c4dd 2dd067e7a439ac5a433bb7aa7eec6b7b  20170106   
                                 a4a773531e35757badfe759f6d69e7a7  20170106   
                                 aa6dab080812d9e8b8e6b8f07c7a399d  20170106   
1faf0ce0b98e02e1568702f516f01a78 806660cb47633263a24bbc53238a9a53  20170106   
25e756b60e551802a441a50e794eff87 21f099b73c49eb58c49371abe5cfe63e  20170106   
2956ff6e5c89beb4bd9e9590ba5596b4 67197f3b5830a66b117539268da239eb  20170106   
32426aebb390a2561d793461a58b0ee8 1e16901cf325d640453b0d9279098803  20170106   
                                 475774a85b02f70fbd08face24d6cec8  20170106   
                                 e6ea7e407888ec088e91bafe475c91d0  20170106   
32d8926c81149eb84ca5af5031a668a0 806660cb47633263a24bbc53238a9a53  20170106   
335e967c2d1836de8e059d929c1a1eb3 58496b4becdf567f1b9f02943a6489a2  20170106   
                                 993c5767292cd8922c422b348d582a5c  20170106   
34a84047a6de70021be428b4ee3152bc e6ea7e407888ec088e91bafe475c91d0  20170106   
3ec691b9d2b5d53ef965fe59b1900b30 385114825a85d6878e7a0978f9ba5546  20170106   
42679123b0cc545eea930f1406704810 ef2f8cc077424405c8d09031ad76ef92  20170106   
435d41ae019cb8db785483793859c9a8 a4d9b88c7ed63d723c70b358a857719c  20170106   
4a48b6f152717235733e4bd48e5a3449 dad5c43748b229667804b2bcd8a40bf5  20170106   
                                 e6ea7e407888ec088e91bafe475c91d0  20170106   
4a9937fc586eddbdee50af81dec74364 66674d25b31f8b9e39389c3dcc94c9a0  20170106   
...                                                                     ...   
b0f1608f6f08dc67df769bd755fb1814 c0883669ac53bc724949ee0cf54457a9  20170106   
                                 dfaf22b3245593f379be6579faa7535f  20170106   
b2537964c1c6812227164f2d21e4f056 2dd067e7a439ac5a433bb7aa7eec6b7b  20170106   
                                 f88d90265ada11b808fc25417e78e3b5  20170106   
b886b6f41c5e9667793644db95a6fe6a e6ea7e407888ec088e91bafe475c91d0  20170106   
b8fd5e70eb720b258b6ad484d382d055 e6ea7e407888ec088e91bafe475c91d0  20170106   
                                 ef2f8cc077424405c8d09031ad76ef92  20170106   
ba53135c9063649e4a25138e81606198 82a7154f05d54bd1a29ec73b76289870  20170106   
c053cb4e67c4dd27bc82ec66a8f4118a 1fe4df4a97ac85e903edf8009b10879a  20170106   
c91bd3afdb4485660be836b73e534873 21f099b73c49eb58c49371abe5cfe63e  20170106   
ca601ff304bf0c0836f8a59f7751b8de 21526ec273ad7b6d230822f32d685fd1  20170106   
cc298af21bdb025746e0ee98148091ba ef2f8cc077424405c8d09031ad76ef92  20170106   
d00fdd18d39a97eb4e773ffebda87912 be2885a06706b121ae3b731d416c0160  20170106   
d474ce5957c78efaa070f8266791060c e6ea7e407888ec088e91bafe475c91d0  20170106   
d4984df3c20408ce64fdfed2ebd62fd2 a4a773531e35757badfe759f6d69e7a7  20170106   
                                 e6ea7e407888ec088e91bafe475c91d0  20170106   
                                 ef2f8cc077424405c8d09031ad76ef92  20170106   
dc93d200565e5bf6e6fc55adb1c5ba06 e231185629db5a1bc134cd088ff190e5  20170106   
                                 febe578c5

In [573]:
#Assigning the epoch time to all the videos seen by a user, if a video has been seen multiple times we
#assign the latest epoch time
user_video_mapper = dict()
def assign_epoch(row):
    user_id = 'user_' + row['UserId']
    video_id = 'video_' + row['VideoId']
    epoch = row['epoch']
    
    if user_id not in user_video_mapper:
        user_video_mapper[user_id] = dict()
    
    video_info_of_user = user_video_mapper[user_id]
    
    if video_id not in video_info_of_user or video_info_of_user[video_id] < epoch:
        video_info_of_user[video_id] = epoch
    
    user_video_mapper[user_id] = video_info_of_user
    
df.apply(assign_epoch, axis=1)
user_video_mapper

{'user_0280dfdd112732a3ac12b12dc770b7af': {'video_36a27b379622f342ec87f9aafadb8f94': 0.0},
 'user_435d41ae019cb8db785483793859c9a8': {'video_a4d9b88c7ed63d723c70b358a857719c': 0.000132100396513124},
 'user_1faf0ce0b98e02e1568702f516f01a78': {'video_806660cb47633263a24bbc53238a9a53': 0.00018494055439077783,
  'video_e6ea7e407888ec088e91bafe475c91d0': 0.7634346103041025},
 'user_3ec691b9d2b5d53ef965fe59b1900b30': {'video_385114825a85d6878e7a0978f9ba5546': 0.000264200793026248},
 'user_ff230d487a7139b65f33b54a4cbd2d9e': {'video_368a480ec0ae105aee8320dd93483e39': 0.00031704095090390183,
  'video_99ca08ae5c8f6d778de6e1d7c1addc17': 0.007186261558672413,
  'video_6b675d876182201fa22887e568c956b7': 0.008216644650019589},
 'user_13d9f55e297d7d9c219fbe9085e1006c': {'video_8b6076b28243980a22f496bb127f3aec': 0.0005019815052946797,
  'video_8c52284238a88f0aa0f264e6c83981bb': 0.043064729194156826,
  'video_1b1e816f38dc32c5cff9833f718582b5': 0.23233817701475346,
  'video_cbe2a10e030ed4ee4c7b877c8db41

In [574]:
# Taking the transpose, so we get the userId's vs VideoId's matrix
user_video_df = pd.DataFrame(user_video_mapper).T

In [575]:
user_video_df.head()

video_0039f67ba5c9538b04f96e8c0cfd5119  \
user_0280dfdd112732a3ac12b12dc770b7af                                     NaN   
user_435d41ae019cb8db785483793859c9a8                                     NaN   
user_1faf0ce0b98e02e1568702f516f01a78                                     NaN   
user_3ec691b9d2b5d53ef965fe59b1900b30                                     NaN   
user_ff230d487a7139b65f33b54a4cbd2d9e                                     NaN   

                                       video_004cf12035df1e5018577003cf61bfec  \
user_0280dfdd112732a3ac12b12dc770b7af                                     NaN   
user_435d41ae019cb8db785483793859c9a8                                     NaN   
user_1faf0ce0b98e02e1568702f516f01a78                                     NaN   
user_3ec691b9d2b5d53ef965fe59b1900b30                                     NaN   
user_ff230d487a7139b65f33b54a4cbd2d9e                                     NaN   

                                       video_00b009802fda66ab98ac10a5935200aa  \
user_0280dfdd112732a3ac12b12dc770b7af                                     NaN   
user_435d41ae019cb8db785483793859c9a8                                     NaN   
user_1faf0ce0b98e02e1568702f516f01a78                                     NaN   
user_3ec691b9d2b5d53ef965fe59b1900b30                                     NaN   
user_ff230d487a7139b65f33b54a4cbd2d9e                                     NaN   

                                       video_00feba0aa20aa7f5a388ef6196025051  \
user_0280dfdd112732a3ac12b12dc770b7af                                     NaN   
user_435d41ae019cb8db785483793859c9a8                                     NaN   
user_1faf0ce0b98e02e1568702f516f01a78                                     NaN   
user_3ec691b9d2b5d53ef965fe59b1900b30                                     NaN   
user_ff230d487a7139b65f33b54a4cbd2d9e                                     NaN   

                                       video_01342e1352689b40c543e1bc5a786d16  \
user_0280dfdd112732a3ac12b12dc770b7af                                     NaN   
user_435d41ae019cb8db785483793859c9a8                                     NaN   
user_1faf0ce0b98e02e1568702f516f01a78                                     NaN   
user_3ec691b9d2b5d53ef965fe59b1900b30                                     NaN   
user_ff230d487a7139b65f33b54a4cbd2d9e                                     NaN   

                                       video_013b09350e350e647f1ded9195f0cd59  \
user_0280dfdd112732a3ac12b12dc770b7af                                     NaN   
user_435d41ae019cb8db785483793859c9a8                                     NaN   
user_1faf0ce0b98e02e1568702f516f01a78                                     NaN   
user_3ec691b9d2b5d53ef965fe59b1900b30                                     NaN   
user_ff230d487a7139b65f33b54a4cbd2d9e                                     NaN   

                                       video_01dce3ef3eba854e472ddd6639543d9a  \
user_0280dfdd112732a3ac12b12dc770b7af                                     NaN   
user_435d41ae019cb8db785483793859c9a8                                     NaN   
user_1faf0ce0b98e02e1568702f516f01a78                                     NaN   
user_3ec691b9d2b5d53ef965fe59b1900b30                                     NaN   
user_ff230d487a7139b65f33b54a4cbd2d9e                                     NaN   

                                       video_022f812c78d65aa855295370f34786ff  \
user_0280dfdd112732a3ac12b12dc770b7af                                     NaN   
user_435d41ae019cb8db785483793859c9a8                                     NaN   
user_1faf0ce0b98e02e1568702f516f01a78                                     NaN   
user_3ec691b9d2b5d53ef965fe59b1900b30                                     NaN   
user_ff230d487a7139b65f33b54a4cbd2d9e                                     NaN   

                                       video_025842be69df5eff0c6e27286cf0bc7e  \
user_0280dfdd112732a3ac12b12dc770b7af                         

In [576]:
#Replacing NaN values by 3, which signifies that a user has not watched the video
#If a user has watched a video, we assign the scaled epoch as the value
user_video_df = user_video_df.fillna(3)

In [588]:
user_video_df.tail(10)

video_0039f67ba5c9538b04f96e8c0cfd5119  \
user_f89dc49cd934761807b4119ca4f876d6                                     3.0   
user_5058a2965cfccec75ac11542c74b2df2                                     3.0   
user_c22edd84e441df56bf844d3d846122ff                                     3.0   
user_fc2b65de0e969cf520f5475ea6e3fcb4                                     3.0   
user_7d26bb559cdc18f41649d1894053aa6c                                     3.0   
user_2ea9f9f2454ba0524e4a76b57f80af1d                                     3.0   
user_c9e5c521ce9559224afcc6b275fa9480                                     3.0   
user_fc51533ec4ca555421893f66fd7b6239                                     3.0   
user_e3b80781e7e3b37f2c17f828750ffb08                                     3.0   
user_4a12f9cc6d8ba6b340a761e238779ea4                                     3.0   

                                       video_004cf12035df1e5018577003cf61bfec  \
user_f89dc49cd934761807b4119ca4f876d6                                     3.0   
user_5058a2965cfccec75ac11542c74b2df2                                     3.0   
user_c22edd84e441df56bf844d3d846122ff                                     3.0   
user_fc2b65de0e969cf520f5475ea6e3fcb4                                     3.0   
user_7d26bb559cdc18f41649d1894053aa6c                                     3.0   
user_2ea9f9f2454ba0524e4a76b57f80af1d                                     3.0   
user_c9e5c521ce9559224afcc6b275fa9480                                     3.0   
user_fc51533ec4ca555421893f66fd7b6239                                     3.0   
user_e3b80781e7e3b37f2c17f828750ffb08                                     3.0   
user_4a12f9cc6d8ba6b340a761e238779ea4                                     3.0   

                                       video_00b009802fda66ab98ac10a5935200aa  \
user_f89dc49cd934761807b4119ca4f876d6                                     3.0   
user_5058a2965cfccec75ac11542c74b2df2                                     3.0   
user_c22edd84e441df56bf844d3d846122ff                                     3.0   
user_fc2b65de0e969cf520f5475ea6e3fcb4                                     3.0   
user_7d26bb559cdc18f41649d1894053aa6c                                     3.0   
user_2ea9f9f2454ba0524e4a76b57f80af1d                                     3.0   
user_c9e5c521ce9559224afcc6b275fa9480                                     3.0   
user_fc51533ec4ca555421893f66fd7b6239                                     3.0   
user_e3b80781e7e3b37f2c17f828750ffb08                                     3.0   
user_4a12f9cc6d8ba6b340a761e238779ea4                                     3.0   

                                       video_00feba0aa20aa7f5a388ef6196025051  \
user_f89dc49cd934761807b4119ca4f876d6                                     3.0   
user_5058a2965cfccec75ac11542c74b2df2                                     3.0   
user_c22edd84e441df56bf844d3d846122ff                                     3.0   
user_fc2b65de0e969cf520f5475ea6e3fcb4                                     3.0   
user_7d26bb559cdc18f41649d1894053aa6c                                     3.0   
user_2ea9f9f2454ba0524e4a76b57f80af1d                                     3.0   
user_c9e5c521ce9559224afcc6b275fa9480                                     3.0   
user_fc51533ec4ca555421893f66fd7b6239                                     3.0   
user_e3b80781e7e3b37f2c17f828750ffb08                                     3.0   
user_4a12f9cc6d8ba6b340a761e238779ea4                                     3.0   

                                       video_01342e1352689b40c543e1bc5a786d16  \
user_f89dc49cd934761807b4119ca4f876d6                                     3.0   
user_5058a2965cfccec75ac11542c74b2df2                                     3.0   
user_c22edd84e441df56bf844d3d846122ff                                     3.0   
user_fc2b65de0e969cf520f5475ea6e3fcb4                                     3.0   
user_7d26bb559cdc18f41649d1894053aa6c                             

# Cosine Similarity Matrix

In [578]:
#Calculating the cosine similarity
user_user_cosine_similarity_df = pd.DataFrame(cosine_similarity(user_video_df))
user_user_cosine_similarity_df.head()

0         1         2         3         4         5         6     \
0  1.000000  0.998551  0.998147  0.998551  0.997106  0.995762  0.998551   
1  0.998551  1.000000  0.998147  0.998551  0.997106  0.995762  0.998551   
2  0.998147  0.998147  1.000000  0.998147  0.996700  0.995353  0.998147   
3  0.998551  0.998551  0.998147  1.000000  0.997106  0.995763  0.998551   
4  0.997106  0.997106  0.996700  0.997106  1.000000  0.994303  0.997106   

       7         8         9       ...         2420      2421      2422  \
0  0.992454  0.995608  0.996913    ...     0.998951  0.998951  0.998952   
1  0.992454  0.995608  0.996914    ...     0.998952  0.998952  0.998952   
2  0.993431  0.995988  0.997708    ...     0.998548  0.998548  0.998549   
3  0.992455  0.995608  0.996914    ...     0.998952  0.998952  0.998952   
4  0.990978  0.994146  0.995460    ...     0.997507  0.997507  0.997508   

       2423      2424      2425      2426      2427      2428      2429  
0  0.998952  0.998306  0.998952  0.998953  0.998629  0.998630  0.998953  
1  0.998952  0.998306  0.998953  0.998953  0.998630  0.998630  0.998953  
2  0.998549  0.998623  0.998549  0.999517  0.998225  0.998226  0.998550  
3  0.998952  0.998306  0.998953  0.998953  0.998630  0.998630  0.998953  
4  0.997508  0.996857  0.997508  0.997509  0.997183  0.997184  0.997509  

[5 rows x 2430 columns]

In [590]:
#Setting the index and the columns headers both as the userId's
user_user_cosine_similarity_df.columns = user_video_df.index
user_user_cosine_similarity_df.head()

user_0280dfdd112732a3ac12b12dc770b7af  \
0                               1.000000   
1                               0.998551   
2                               0.998147   
3                               0.998551   
4                               0.997106   

   user_435d41ae019cb8db785483793859c9a8  \
0                               0.998551   
1                               1.000000   
2                               0.998147   
3                               0.998551   
4                               0.997106   

   user_1faf0ce0b98e02e1568702f516f01a78  \
0                               0.998147   
1                               0.998147   
2                               1.000000   
3                               0.998147   
4                               0.996700   

   user_3ec691b9d2b5d53ef965fe59b1900b30  \
0                               0.998551   
1                               0.998551   
2                               0.998147   
3                               1.000000   
4                               0.997106   

   user_ff230d487a7139b65f33b54a4cbd2d9e  \
0                               0.997106   
1                               0.997106   
2                               0.996700   
3                               0.997106   
4                               1.000000   

   user_13d9f55e297d7d9c219fbe9085e1006c  \
0                               0.995762   
1                               0.995762   
2                               0.995353   
3                               0.995763   
4                               0.994303   

   user_ca601ff304bf0c0836f8a59f7751b8de  \
0                               0.998551   
1                               0.998551   
2                               0.998147   
3                               0.998551   
4                               0.997106   

   user_f717dc3f757dc18cc0c3f27ed3e94ef8  \
0                               0.992454   
1                               0.992454   
2                               0.993431   
3                               0.992455   
4                               0.990978   

   user_0d2717ba754737c78430788ed77a1db7  \
0                               0.995608   
1                               0.995608   
2                               0.995988   
3                               0.995608   
4                               0.994146   

   user_087c741c167a352d07822fcab58046e5  \
0                               0.996913   
1                               0.996914   
2                               0.997708   
3                               0.996914   
4                               0.995460   

                   ...                    \
0                  ...                     
1                  ...                     
2                  ...                     
3                  ...                     
4                  ...                     

   user_f89dc49cd934761807b4119ca4f876d6  \
0                               0.998951   
1                               0.998952   
2                               0.998548   
3                               0.998952   
4                               0.997507   

   user_5058a2965cfccec75ac11542c74b2df2  \
0                               0.998951   
1                               0.998952   
2                               0.998548   
3                               0.998952   
4                               0.997507   

   user_c22edd84e441df56bf844d3d846122ff  \
0                               0.998952   
1                               0.998952   
2                               0.998549   
3                               0.998952   
4                               0.997508   

   user_fc2b65de0e969cf520f5475ea6e3fcb4  \
0                               0.998952   
1                               0.998952   
2                               0.998549   
3                               0.998952   
4                               0.997508   

   user_7d26bb559cdc18f41649

In [591]:
user_user_cosine_similarity_df.set_index(user_video_df.index)

user_0280dfdd112732a3ac12b12dc770b7af  \
user_0280dfdd112732a3ac12b12dc770b7af                               1.000000   
user_435d41ae019cb8db785483793859c9a8                               0.998551   
user_1faf0ce0b98e02e1568702f516f01a78                               0.998147   
user_3ec691b9d2b5d53ef965fe59b1900b30                               0.998551   
user_ff230d487a7139b65f33b54a4cbd2d9e                               0.997106   
user_13d9f55e297d7d9c219fbe9085e1006c                               0.995762   
user_ca601ff304bf0c0836f8a59f7751b8de                               0.998551   
user_f717dc3f757dc18cc0c3f27ed3e94ef8                               0.992454   
user_0d2717ba754737c78430788ed77a1db7                               0.995608   
user_087c741c167a352d07822fcab58046e5                               0.996913   
user_9151ab2a901458f27e36548d4a8dc011                               0.997303   
user_66d4abf4f3b733c27591b3f47c85eff7                               0.997826   
user_04c1e14fd54a06c0d4fe98cf0cb8b04c                               0.998551   
user_4a48b6f152717235733e4bd48e5a3449                               0.996787   
user_0d164b3342b7c442b8a5cfc8a608bee1                               0.992154   
user_d474ce5957c78efaa070f8266791060c                               0.997781   
user_6f1e17087d0349318105784ce0fb323a                               0.993914   
user_42679123b0cc545eea930f1406704810                               0.998058   
user_d4984df3c20408ce64fdfed2ebd62fd2                               0.998077   
user_6d51418c537c27943c2385c0813004ac                               0.996216   
user_535959f23c9908bd8cabd72291a814f0                               0.998552   
user_e8df3764a62d0cb144f3095f56dcfebd                               0.998552   
user_32426aebb390a2561d793461a58b0ee8                               0.997103   
user_b0f1608f6f08dc67df769bd755fb1814                               0.994575   
user_8d114c338b475f24b7f75fdfa30e0f74                               0.998552   
user_1f9be9226d4393fff4b8d6b94e54c4dd                               0.997102   
user_5a09eef63fca4f68a310d3795f7409ba                               0.994587   
user_fbdb007a5d3983d11a6a15d3feeeb253                               0.997106   
user_deed7490952a6746610ee2ee9fd219b7                               0.998552   
user_544bcf117ab3fbf100285b95d8639e8d                               0.995576   
...                                                                      ...   
user_3389a5feb923df751c5c44e51d49ff51                               0.998294   
user_739429126166eb7e3e82edd97e3bd06a                               0.997975   
user_db4ef41ee181a3611f7ca39566f4793a                               0.997642   
user_07558b2c2e49b174f610c2331557c9bd                               0.998949   
user_5eaaa5bad58b26d3ecd5b91e05a13e36                               0.998949   
user_c0969630e358ea2aca6608f9340a4dce                               0.998950   
user_bbe382de361d15f70ffe32d5f1409a8d                               0.998950   
user_abd59a13a938834d521d8ce0f1d637e6                               0.998950   
user_0b086b41502f3e9a636ea3c670119ca1                               0.998625   
user_a4ed80a78b0d6d0532482164040a6ff9                               0.998950   
user_7dcfe00b296e8fa6737eec27f50f859c                               0.998299   
user_31194bb7d4ba9fab15c4917be3232406                               0.998950   
user_fa3ce952417cc24ca57d0d058c1aff9e                               0.997974   
user_ffe24976e4928be8a80f8675411f6729                               0.998951   
user_8d32ef0a17800604f28f6420a402c017                               0.998951   
user_c40b25089b13e3feefda4e719e665854                               0.998300   
user_faa9ae71e9a3e8b9bc6c3c8bfe0ae8f8                               0.998629   
user_540339a721e2e514f2a5c3cd9955a8e3                               0.998951   
user_a427781c2d437b846107ab61f031434b  

# Recommendation Engine

In [580]:
#Gives list of similar users to a particular user
def get_similarity(user_id):
    similarity = pd.DataFrame(user_user_cosine_similarity_df[user_id])
    similarity["username"] = list(user_user_cosine_similarity_df)
    similarity = similarity.sort_values(by=user_id,ascending=False)
    similarity = similarity[similarity[user_id]<0.99]
    return similarity.head(10)

In [581]:
#Once the list of similar users is obtained we get the dataframe containing the videos watched 
#by the similar users
def get_recommended_users(similar_users):
    rec_videos = pd.DataFrame()
    target_user = list(similar_users)[0]
    for user in similar_users['username'] :
        rec_videos[user]=user_video_df.loc[user]
    return rec_videos.T

In [582]:
#Getting the recommended videos for a user
def get_videos(df_similar_users) :
    videos = []
    for user in df_similar_users.index:
        for video in list(df_similar_users):
            if df_similar_users.loc[user,video]>=epoch_start_date and df_similar_users.loc[user,video]<=epoch_end_date and df_similar_users.loc[user,video]<3:
                videos.append(video)
    return list(set(videos))[:3]

In [ ]:
#Getting the recommendations for every user using the above defined functions
recommended_videos = {}
for user in user_video_df.index:
    similar_users = get_similarity(user)
    df_similar_users = get_recommended_users(similar_users)
    rec_videos = get_videos(df_similar_users)
    recommended_videos[user]=rec_videos

# Final Recommendations for the users

In [592]:
recommended_videos

{'user_0280dfdd112732a3ac12b12dc770b7af': ['video_79efbb9a91735dafba560a9eb470ac4a',
  'video_7161a5300b13ccd70e51e3891d51e8dd',
  'video_bbc40289bb0deaeff9a0df91db2fbd88'],
 'user_435d41ae019cb8db785483793859c9a8': ['video_79efbb9a91735dafba560a9eb470ac4a',
  'video_7161a5300b13ccd70e51e3891d51e8dd',
  'video_bbc40289bb0deaeff9a0df91db2fbd88'],
 'user_1faf0ce0b98e02e1568702f516f01a78': ['video_79efbb9a91735dafba560a9eb470ac4a',
  'video_7161a5300b13ccd70e51e3891d51e8dd',
  'video_bbc40289bb0deaeff9a0df91db2fbd88'],
 'user_3ec691b9d2b5d53ef965fe59b1900b30': ['video_7161a5300b13ccd70e51e3891d51e8dd',
  'video_bbc40289bb0deaeff9a0df91db2fbd88',
  'video_e33431ce046729169771d519df4930c6'],
 'user_ff230d487a7139b65f33b54a4cbd2d9e': ['video_6bdb9aab915ae9bc0f783e86b083b687',
  'video_f3f18d47b0a2ad5bb2fa26382acc1206',
  'video_7161a5300b13ccd70e51e3891d51e8dd'],
 'user_13d9f55e297d7d9c219fbe9085e1006c': ['video_6bdb9aab915ae9bc0f783e86b083b687',
  'video_c86b71b30b685ac003c4ffcfd03a7a71',
 